In [1]:
import numpy as np

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update parameters and return input gradient
        pass


class Dense(Layer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = np.array(input, dtype=np.float64)
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        output_gradient = np.array(output_gradient, dtype=np.float64)
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient


class Activation(Layer):
    def __init__(self, activation, activation_prime):
        super().__init__()
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))


class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)


class Activation_Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

class Softmax(Activation):
    def __init__(self):
        def softmax(x):
            return 1 / (1 + np.exp(-x))

        def softmax_prime(x):
            s = softmax(x)
            return s * (1 - s)

        super().__init__(softmax, softmax_prime)


class ReLu(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0, x)

        def relu_prime(x):
            return np.where(x <= 0, 0, 1)

        super().__init__(relu, relu_prime)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))


def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)


def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))


def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output


def test(network, input_data, output_data, loss):
    error = 0
    predictions = []
    for x, y in zip(input_data, output_data):
        # forward
        output = predict(network, x)
        predictions.append(output)  # Collect predictions
        # error
        error += loss(y, output)

    error /= len(input_data)
    predictions = np.array(predictions)
    return error, predictions


def train(network, loss, loss_prime, x_train, y_train, epochs=1000, learning_rate=0.01, verbose=True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)
            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")


In [2]:
import numpy as np
from scipy import signal

class Flatten(Layer):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)


class ConvolutionalLayer(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        super().__init__()
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient


class MaxPooling(Layer):
    def __init__(self, pool_size):
        super().__init__()
        self.pool_size = pool_size

    def forward(self, input):
        self.input = input
        depth, height, width = input.shape
        pooled_height = height // self.pool_size
        pooled_width = width // self.pool_size
        self.output = np.zeros((depth, pooled_height, pooled_width))

        for d in range(depth):
            for ph in range(pooled_height):
                for pw in range(pooled_width):
                    h_start = ph * self.pool_size
                    h_end = h_start + self.pool_size
                    w_start = pw * self.pool_size
                    w_end = w_start + self.pool_size
                    window = input[d, h_start:h_end, w_start:w_end]
                    self.output[d, ph, pw] = np.max(window)

        return self.output

    def backward(self, output_gradient, learning_rate):
        depth, pooled_height, pooled_width = output_gradient.shape
        input_gradient = np.zeros_like(self.input)

        for d in range(depth):
            for ph in range(pooled_height):
                for pw in range(pooled_width):
                    h_start = ph * self.pool_size
                    h_end = h_start + self.pool_size
                    w_start = pw * self.pool_size
                    w_end = w_start + self.pool_size
                    window = self.input[d, h_start:h_end, w_start:w_end]
                    max_value = np.max(window)
                    mask = (window == max_value)
                    input_gradient[d, h_start:h_end, w_start:w_end] += mask * output_gradient[d, ph, pw]

        return input_gradient


In [4]:

from keras.datasets import mnist
from keras.utils import to_categorical

def preprocess_data(x, y, limit):
    x = x.reshape(x.shape[0], 1,  28, 28)
    x = x.astype("float32") / 255
    y = to_categorical(y)
    y = y.reshape(y.shape[0], 10, 1)
    return x[:limit], y[:limit]

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, y_train.shape[0])
x_test, y_test = preprocess_data(x_test, y_test, y_test.shape[0])
print(y_train.shape[0], y_test.shape[0])

60000 10000


In [2]:
epochs = 7
learning_rate = 0.15
dense = 32
kernel1 = kernels1 = kernel2 = 5
kernels2 = 3

b = (28-kernel1+1)//2
c = b-kernel2+1
network = [
    ConvolutionalLayer((1, 28, 28), kernel1, kernels1),
    Activation_Sigmoid(),
    MaxPooling(2),
    ConvolutionalLayer((kernels1, b, b), kernel2, kernels2),
    Activation_Sigmoid(),
    Flatten((kernels2, c, c), (kernels2 * c * c, 1)),
    Dense(kernels2 * c * c, dense), Activation_Sigmoid(), Dense(dense, 10), Activation_Sigmoid()
]
train(network, binary_cross_entropy, binary_cross_entropy_prime,
    x_train, y_train, epochs=epochs, learning_rate=learning_rate)

error, predictions = test(network, x_test, y_test, binary_cross_entropy)
print(f"error = {error}")

1/4, error=0.11078565230551378
2/4, error=0.052571209873177496
3/4, error=0.04097422386385086
4/4, error=0.0352607173600701
error = 0.03345366608587281


In [5]:
import numpy as np

def precision_recall(y_true, y_pred, num_classes):
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)

    for i in range(num_classes):
        true_positives = np.sum((y_pred == i) & (y_true == i))
        false_positives = np.sum((y_pred == i) & (y_true != i))
        false_negatives = np.sum((y_pred != i) & (y_true == i))

        if true_positives + false_positives > 0:
            precision[i] = true_positives / (true_positives + false_positives)
        else:
            precision[i] = 0.0

        if true_positives + false_negatives > 0:
            recall[i] = true_positives / (true_positives + false_negatives)
        else:
            recall[i] = 0.0

    return precision, recall
from sklearn.metrics import roc_curve, auc

def calculate_roc_auc(y_true, y_scores, num_classes):
  roc_auc = np.zeros(num_classes)

  for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_true == i, y_scores[:, i])
    roc_auc[i] = auc(fpr, tpr)

  return roc_auc

In [6]:
y_pred = np.argmax(predictions, axis=1)

# Преобразуем метки из one-hot encoding в класс
y_true = np.argmax(y_test, axis=1)

# Расчет Precision и Recall
precision, recall = precision_recall(y_true, y_pred, num_classes=10)
print("Precision per class: ", precision)
print("Recall per class: ", recall)

# Расчет ROC-AUC
y_scores = np.array([predict(network, x) for x in x_test]).squeeze()
roc_auc = calculate_roc_auc(y_true, y_scores, num_classes=10)
print("ROC-AUC per class: ", roc_auc)

Precision per class:  [0.97049847 0.95989761 0.96318408 0.93450635 0.97257384 0.92560175
 0.96153846 0.9637827  0.95592865 0.91873805]
Recall per class:  [0.97346939 0.99118943 0.9379845  0.94653465 0.9389002  0.94843049
 0.96555324 0.93190661 0.93531828 0.95242815]
ROC-AUC per class:  [0.99824234 0.99958506 0.99265357 0.99394905 0.99796639 0.99600657
 0.997901   0.99558505 0.99476984 0.99464325]


In [5]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_roc_curves(y_true, y_scores, num_classes):
    plt.figure()
    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true == i, y_scores[:, i])
        plt.plot(fpr, tpr, label=f'Class {i} (area = {auc(fpr, tpr):.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves')
    plt.legend(loc='lower right')
    plt.show()
    
plot_roc_curves(y_true, y_scores, num_classes=10)